In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv('student_exam_scores_12_13.csv')
print(df.head())

  student_id  hours_studied  sleep_hours  attendance_percent  Internal_marks  \
0       S001            8.0          8.8                72.1              45   
1       S002            1.3          8.6                60.7              55   
2       S003            4.0          8.2                73.7              86   
3       S004            3.5          4.8                95.1              66   
4       S005            9.1          6.4                89.8              71   

   exam_score  
0        30.2  
1        25.0  
2        35.8  
3        34.0  
4        40.3  


In [3]:
print(df.isnull().sum())

student_id            0
hours_studied         0
sleep_hours           0
attendance_percent    0
Internal_marks        0
exam_score            0
dtype: int64


In [71]:
X=df[['hours_studied',"Internal_marks","attendance_percent"]].values
y=df['exam_score'].values
# add bias term
bias = np.ones((X.shape[0], 1))
X = np.hstack((bias, X))


In [73]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [75]:
class LinearRegression:
    def __init__(self):
        self.weights=None
    def fit(self,X,y):
        XTX=X.T @X
        XTY=X.T @y
        self.weights=np.linalg.inv(XTX) @XTY
    def predict(self,X):
        return X @ self.weights

    def MSE(self,y_true,y_pred):
        return np.mean((y_true-y_pred)**2)

    def r2(self,y_true,y_pred):
        num=np.sum((y_true-y_pred)**2)
        den=np.sum((y_true-np.mean(y_true))**2)
        return (1- num/den)
    

In [77]:
model=LinearRegression()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)

In [79]:
mse=model.MSE(y_test,y_pred)
r2=model.r2(y_test,y_pred)
print("mse",mse)
print("r2",r2)

mse 11.047991087183163
r2 0.7918226819398652


In [82]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5,shuffle=True,random_state=42)
mse=[]
r2=[]
for train_idx,test_idx in kf.split(X):
    X_train,X_test=X[train_idx],X[test_idx]
    y_train,y_test=y[train_idx],y[test_idx]
    model=LinearRegression()
    model.fit(X_train,y_train)
    y_predict=model.predict(X_test)
    mses=model.MSE(y_test,y_predict)
    r2s=model.r2(y_test,y_predict)
    mse.append(mses)
    r2.append(r2s)

for i in range(5):
    print(f"Fold: {i+1} , MSE: {mse[i]} , R2: {r2[i]}")

Fold: 1 , MSE: 11.04799108718302 , R2: 0.7918226819398678
Fold: 2 , MSE: 8.499078016261143 , R2: 0.8099605076614791
Fold: 3 , MSE: 10.413774929672812 , R2: 0.7634484274585929
Fold: 4 , MSE: 8.294196653644175 , R2: 0.7805140615458572
Fold: 5 , MSE: 8.4991800961076 , R2: 0.8252429979452228


In [84]:
print("Avg MSE",np.mean(mse))
print("AVG R2",np.mean(r2))

Avg MSE 9.35084415657375
AVG R2 0.794197735310204
